# 準備
## ライブラリのインストール
Colaboratory環境で実行する場合は`japanaize_matplotlib`をインストールする必要があります。

SageMaker環境で以下を実行する必要はありませんが、実行しても害はありません。

In [1]:
%pip install japanize_matplotlib

Note: you may need to restart the kernel to use updated packages.


## ライブラリのインポートとデータの読み込み

この資料で使用するライブラリを一括してインポートし、必要なデータを読み込みます。

SageMakerやColaboratoryの接続がタイムアウトした時は、
以下のセルを再実行して下さい。

In [ ]:
#ライブラリーのインポート
import sklearn
import numpy as np
from sklearn import manifold
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import japanize_matplotlib
import math
import random
import pandas as pd

#データセットの読み出し
akutagawa_name = np.load('data/akutagawa_name.npy', allow_pickle=True)
kikuchi_name = np.load('data/kikuchi_name.npy', allow_pickle=True)
hgram = np.load('data/hgram.npy', allow_pickle=True)
words = np.load('data/words.npy', allow_pickle=True)
print('芥川龍之介の作品は以下の20編')
print(' '.join(akutagawa_name))
print('\n菊池寛の作品は以下の20編')
print(' '.join(kikuchi_name))
print('\n40編の作品に現れる語の総数は{}語'.format(len(words)))

## 2.9 データの加工(第2ラウンド)
これまでの方法で正解率が高くない原因を、データの加工という観点から考えてみます。\
まずはじめに、第1ラウンドで使用した加工済みの「歯車(芥川)」と「蜘蛛の糸(芥川)」の頻度分布を比較してみます。

In [ ]:
 # 歯車(芥川)と蜘蛛の糸(芥川)の頻度分布の比較
def count(n, w):
    if w in hgram[n]:
        return hgram[n][w]
    else:
        return 0

_, axes = plt.subplots(1, 2, figsize=[15,5])

y = [count(5, w) for w in words] # countはn(第一引数)番目の作品にwという単語は何回出現しているかを返す関数、wordsは40作品に出現するすべての単語を持つリスト。
axes[0].plot(range(len(words)), y)
axes[0].set_title('歯車', fontsize=20)
axes[0].set_ylim([0, 160])
axes[0].xaxis.set_visible(False)
axes[0].set_yticklabels(range(0, 161, 20), fontsize=16)

y = [count(19, w) for w in words] #5は歯車に、19は蜘蛛の糸に対応した番号
axes[1].plot(range(len(words)), y)
axes[1].set_title('蜘蛛の糸', fontsize=20)
axes[1].set_ylim([0, 160])
axes[1].xaxis.set_visible(False)
axes[1].set_yticklabels(range(0, 161, 20), fontsize=16)

plt.show()


共に芥川龍之介の作品ですが、分布の見た目は大きく異なっています。この相違の最大の原因は作品の長さにあり、芥川か菊池かを判断するための障害になりかねません。



In [ ]:
 # 歯車の語数と蜘蛛の糸の語数の比較
print(f'歯車の名詞・動詞・形容詞・副詞の語数は {sum(hgram[5].values())}')
print(f'蜘蛛の糸の名詞・動詞・形容詞・副詞の語数は {sum(hgram[19].values())}')

実際、「歯車」に現れる名詞・動詞・形容詞・副詞の個数は、「蜘蛛の糸」のそれのそれの10倍以上であることが分かります。

ここでは、正規化をすることで、複数種類の観測値のスケールがそろえられ、より正確な評価を行うことができると考え、作品の頻度分布に対して$L^2$正規化を行い、正解率の改善を目指します。

### $L^2$正規化
$L^2$正規化とは各データのベクトルの長さを1にする処理です。

$$
  \frac{\vec v}{\Vert\vec v\Vert} =
  \left(\frac{v_1}{\sqrt{v_1^2 + \dots + v_{16,301}^2}}, \dots,
    \frac{v_{16,301}}{\sqrt{v_1^2 + \dots + v_{16,301}^2}}\right)
$$

$L^2$正規化した「歯車」（`hgrm[5]`）と「蜘蛛の糸」（`hgrm[19]`）のデータを表示してみましょう。



In [ ]:
 # L2正規化を行う関数を作成&歯車と蜘蛛の糸の(正規化後の)分布を表示

def l2normalize(dct): # L2正規化
    s = np.sqrt(sum([v**2 for v in dct.values()]))
    return dict([(k, v/s) for k, v in dct.items()]) 

hgrm_nrm = [l2normalize(dct) for dct in hgram] # hgramは前に作成したリスト。各作品の単語の種類とその単語の出現回数を持つ辞書型オブジェクトを作品の数だけ持つ。

def countl2(n, w): # n番目の作品にwという単語が何回出現しているかを返す関数
    if w in hgrm_nrm[n]:
        return hgrm_nrm[n][w]
    else:
        return 0

_, axes = plt.subplots(1, 2, figsize=[15,5])

y = [countl2(5, w) for w in words]
axes[0].plot(range(len(words)), y)
axes[0].set_title('歯車', fontsize=20)
axes[0].set_ylim([0, 1])
axes[0].xaxis.set_visible(False)
axes[0].set_yticklabels(np.round(np.arange(0, 1.1, 0.2), 1), fontsize=16)

y = [countl2(19, w) for w in words]
axes[1].plot(range(len(words)), y)
axes[1].set_title('蜘蛛の糸', fontsize=20)
axes[1].set_ylim([0, 1])
axes[1].xaxis.set_visible(False)
axes[1].set_yticklabels(np.round(np.arange(0, 1.1, 0.2), 1), fontsize=16)

plt.show()



>上のプログラムでは`hgrm`の各要素をL2正規化して得られる配列を`hgrm_nrm`とします。
>
>したがって、`hgrm[i]`のL2正規化は`hgrm_nrm[i]`になります。

他の作品に対しても正規化を行います。

In [ ]:
 # 他の作品でもL2正規化&表示
_, axes = plt.subplots(10, 4, figsize=[20,30])
name = list(akutagawa_name) + list(kikuchi_name)
for n in range(40):
    title = name[n]
    y = [countl2(n, w) for w in words]
    ax = axes[int(n/4), n%4]
    ax.plot(range(len(words)), y)
    ax.set_ylim([0, 1])
    ax.set_title(title+('（芥川）' if n < 20 else '（菊池）'), fontsize=20)
    ax.xaxis.set_visible(False)
plt.show()

$L^2$正規化する前と後で比較してみましょう。これで、分布の形状が作品の長さに依存するという問題を解消できました。

In [ ]:
 # 正規化する前
_, axes = plt.subplots(10, 4, figsize=[20,30])
for n in range(40):
    title = name[n]
    y = [count(n, w) for w in words]
    ax = axes[int(n/4), n%4]
    ax.plot(range(len(words)), y)
    ax.set_ylim([0, 400])
    ax.set_title(title+('（芥川）' if n < 20 else '（菊池）'), fontsize=20)
    ax.xaxis.set_visible(False)
plt.show()

## 2.10 学習アルゴリズムの選択(第2ラウンド)
$L^2$正規化したベクトルに対してMDSを行い、16,301次元を3次元に圧縮してみます。k-NNアルゴリズムが有効そうであれば、引き続きk-NNアルゴリズムを使用します。

In [ ]:
# MDSで次元圧縮
def dl2(dictx, dicty):
    sq = 0
    for key in set(dictx.keys()) | set(dicty.keys()):
        x, y = 0, 0
        if key in dictx: 
            x = dictx[key]
        if key in dicty:
            y = dicty[key]
        sq += (x - y)**2
    return math.sqrt(sq)

temp = [] 
for dictx in hgrm_nrm:
    temp.append([dl2(dictx, dicty) for dicty in hgrm_nrm])
dl2_matrix = np.array(temp) # 距離の行列

dl2_matrix

from sklearn import manifold
mds = manifold.MDS(n_components=3, dissimilarity="precomputed", random_state=6)
pos = mds.fit_transform(dl2_matrix)
pos

先ほどと同様に可視化してみましょう。

In [ ]:
 # MDSで3次元に圧縮
l1 = len(akutagawa_name)
l2 = len(kikuchi_name)

fig = go.Figure(
    layout=go.Layout(
    title="L2正規化後、ユークリッド距離に基づくデータの分布（MDSで次元圧縮）" ,
    showlegend=True,
    legend=dict(x=0.7, y=0.99, xanchor='left', yanchor='top', font=dict(size=16))
    )
)

fig.add_trace(go.Scatter3d(
        x = pos[0: l1,0], y = pos[0: l1,1], z = pos[0: l1,2],
        mode = 'markers',
        marker = dict(symbol='cross', color='black', size=5, 
                      line=dict(width=0), opacity=1 ),
        name = '芥川'
        )
)

fig.add_trace(go.Scatter3d(
        x = pos[l1: l1+l2, 0], y = pos[l1: l1+l2, 1], z = pos[l1: l1+l2, 2],
        mode = 'markers',
        marker = dict(symbol='circle', color='black', size=5, 
                      line=dict(width=0), opacity=1),
        name = '菊池'
        )
)



参考程度にしかなりませんが、$L^2$正規化する前と比較して芥川と菊池のデータの凝集具合が改善されたように見えます。\
したがって、k-NNアルゴリズムでの正解率の上昇が期待できます。

以下でも引き続きk-NNアルゴリズムを使用します。

## 2.11 ハイパーパラメータの最適化とモデルの評価(第2ラウンド)

第1ラウンドの場合と同様に、
$k = 1, 2, \dots, 10$に対して、
5分割交差検証により予測の正解率を計算することで、最適なハイパーパラメータの値を求めます。

In [ ]:
# kを変化させて正解率を調べる
fig = plt.figure(figsize=(8,6))

labels = [0]*l1 + [1]*l2

parameters = [{'metric': ['precomputed'], 'n_neighbors': list(range(1,11))}]
clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)
clf.fit(dl2_matrix, labels)

x = []
y = []

params = clf.cv_results_["params"]
mean_test_score = clf.cv_results_["mean_test_score"]
std_test_score = clf.cv_results_["std_test_score"]
for p, m, s in zip(params, mean_test_score, std_test_score):
    print(f"{m:.3f} (+/- {s/2:.3f}) for {p}")
    x.append(p["n_neighbors"])
    y.append(m)


plt.ylim(0, 1)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('$k$', fontsize=16)
plt.ylabel('正解率', fontsize=16)
plt.bar(x, y)
plt.grid()
plt.show()

$k$の最適値と、正解率の最大値は以下の通りです。

In [ ]:
print(f"kの最適値:\t{clf.best_estimator_.n_neighbors}")
scores = model_selection.cross_val_score(clf.best_estimator_, X = dl2_matrix, y = labels, cv = 5)
print(f"k={clf.best_estimator_.n_neighbors}での正解率:\t{np.average(scores)}")

正解率は上がりましたが、未だに満足できる値ではありません。探索を続けます。